In [1]:
import yfinance as yf
import pandas as pd
import datetime
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pandas_datareader import data
from dask.distributed import Client
from dask.distributed import as_completed
import openpyxl
import xlsxwriter
import pandas_datareader.data as web

In [2]:
end = datetime.today()
start = end - relativedelta(years=2)

In [3]:
end.strftime('%Y-%m-%d')

'2022-06-05'

In [4]:
etf_bonds = ['LQD', 'AGG', 'NEAR', 'IUSB', 'ISTB', 'IMTB', 'ILTB', 'GBF']
etf_muni_bonds = ['MUB', 'SUB', 'MEAR']

etf_treasuries = ['AGZ', 'GOVT', 'BIL', 'SHV', 'SHY', 'IEI', 'IEF', 'TLT']

FRED_Indicators = [
"ASPUS",\
"AWHAETP",\
"B432RG3A086NBEA",\
"BAA10Y",\
"BACDINA066MNFRBNY",\
"BACTSAMFRBDAL",\
"BAMLC0A0CM",\
"BAMLCC0A1AAATRIV",\
"BAMLCC0A4BBBTRIV",\
"BAMLH0A3HYC",\
"BAMLHYH0A0HYM2TRIV",\
"BOGMBASE",\
"BOGZ1FA105015103Q",\
"BOGZ1FA145050005Q",\
"BOGZ1FA205050005Q",\
"BOGZ1FA315050005Q",\
"BOGZ1FA385050005Q",\
"BOGZ1FL105015105Q",\
"BUSAPPWNSACA",\
"CASTHPI",\
"CES0500000003",\
"CES4348400001",\
"CFNAI",\
"CFNAIDIFF",\
"CFSBCACTIVITYMFG",\
"CILACBQ158SBOG",\
"CIVPART",\
"COMREPUSQ159N",\
"CP",\
"CPALTT01USQ657N",\
"CPIAUCSL",\
"CSCICP03USM665S",\
"CSUSHPINSA",\
"CUSR0000SEHA",\
"CUUR0000SEHA",\
"DALLCIACBEP",\
"DCOILBRENTEU",\
"DCOILWTICO",\
"DEXCHUS",\
"DEXUSUK",\
"DFF",\
"DFII10",\
"DGORDER",\
"DGS1",\
"DGS10",\
"DGS1MO",\
"DGS2",\
"DGS30",\
"DGS3MO",\
"DGS5",\
"DRBLACBS",\
"DRCCLACBS",\
"DRSFRMACBS",\
"DTB3",\
"DTWEXBGS",\
"EMRATIO",\
"ERENTUSQ176N",\
"ETOTALUSQ176N",\
"FEDFUNDS",\
"FGCCSAQ027S",\
"FPCPITOTLZGUSA",\
"GASREGW",\
"GDPC1",\
"GFDEBTN",\
"GFDEGDQ188S",\
"GOLDAMGBD228NLBM",\
"GVZCLS",\
"HDTGPDUSQ163N",\
"HOSMEDUSM052N",\
"IC4WSA",\
"ICSA",\
"IIPUSNETIQ",\
"INDPRO",\
"INTDSRUSM193N",\
"IRLTLT01USM156N",\
"KCFSI",\
"LES1252881600Q",\
"LEU0252918500Q",\
"LFWA64TTUSM647S",\
"LRUN64TTUSQ156S",\
"LXXRCSA",\
"M1",\
"M1V",\
"M2V",\
"MABMM301USM189S",\
"MANMM101USA189S",\
"MEFAINUSA672N",\
"MEHOINUSA672N",\
"MEPAINUSA672N",\
"MICH",\
"MORTGAGE30US",\
"MPRIME",\
"MSPUS",\
"MZMSL",\
"NFCI",\
"NROU",\
"NYSTHPI",\
"PAYEMS",\
"PERMIT",\
"POP",\
"POPTOTUSA647NWDB",\
"PPIACO",\
"PRFI",\
"PSAVERT",\
"Q10096USQ144NNBR",\
"RECPROUSM156N",\
"RSAHORUSQ156S",\
"SPCS20RSA",\
"STLFSI2",\
"T10Y2Y",\
"T10Y3M",\
"T10YIE",\
"T5YIE",\
"T5YIFR",\
"TB3MS",\
"TCU",\
"TDSP",\
"TEDRATE",\
"TOTALSA",\
"TREAST",\
"TTLHH",\
"TWEXB",\
"UMCSENT",\
"U6RATE",\
"UNRATE",\
"USPHCI",\
"USREC",\
"USROA",\
"USROE",\
"USSLIND",\
"USSTHPI",\
"VIXCLS",\
"VXVCLS",\
"WDFUELLA",\
"WGS3MO",\
"WILLLRGCAPVAL",\
"WILLMICROCAPPR",\
"WPU0911",\
"WTB3MS"    
]

etf_bonds = [*etf_bonds,*etf_muni_bonds,*etf_treasuries]

In [5]:
SP500ticker_list = pd.read_html('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
SP600ticker_list = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_600_companies')[1]
SP400ticker_list = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_400_companies')[0]


In [6]:
sp500 = SP500ticker_list['Symbol']
sp500 = [w.replace('.', '-') for w in sp500]
sp600 = SP600ticker_list['Ticker symbol']
sp600 = [w.replace('.', '-') for w in sp600]
sp400 = SP400ticker_list['Ticker symbol']
sp400 = [w.replace('.', '-') for w in sp400]


In [7]:
len(sp500)+len(sp600)+len(sp400)

1505

In [8]:
print(len(sp500))
SP1500 = list()
SP1500.extend(sp500)
sp600 = list(set(sp600).difference(SP1500))
print(len(sp600))
SP1500.extend(sp600)
sp400 = list(set(sp400).difference(SP1500))
print(len(sp400))
SP1500.extend(set(sp400).difference(SP1500))

504
601
400


In [9]:
sp1500_index_df = pd.DataFrame()

lists = ["sp500","sp600","sp400"]

for l in lists:
    if(l=='sp500'):
        df = pd.DataFrame(sp500,columns=['Symbol'])
        print(len(df))
        #range_ = 
        #print(range_)
        df['Market Index'] = pd.Series(["SP500" for x in range(0,len(df.index),1)])
    elif(l=='sp600'):
        df = pd.DataFrame(np.array(sp600),columns=['Symbol'])        
        print(len(df))        
        df['Market Index'] = pd.Series(["SP600" for x in range(len(df.index))])
        df.index = range(len(sp500),(len(sp500)+len(df.index)),1)        
    else:
        df = pd.DataFrame(np.array(sp400),columns=['Symbol'])
        print(len(df))
        df['Market Index'] = pd.Series(["SP400" for x in range(len(df.index))])
        df.index = range((len(sp500)+len(sp600)),(len(sp500)+len(sp600)+len(df.index)),1)
    sp1500_index_df = pd.concat([sp1500_index_df,df],axis=0)

504
601
400


In [10]:
len(sp1500_index_df)-len(SP1500)

0

In [11]:
def Fred_Data(npa):
    name = npa[0]
    start_ = npa[1]
    end_ = npa[2]
    temp = web.DataReader(str(name), 'fred', start_, end_)
    temp.index = pd.to_datetime(temp.index)
    temp.columns = ['Value']
    temp['Symbol'] = name
    #temp = temp.resample(frequency).mean().dropna()
    return(temp)

def dl(npa):
    name = npa[0]
    start_ = npa[1]
    end_ = npa[2]
    subset = yf.download(name, start=start_, end=end_, auto_adjust=True).iloc[:, :6].dropna(axis=0, how='any')
    subset['Symbol'] = name

    return (subset)
    
def dl2(assets):

    yahoo_financials = YahooFinancials(assets)

    data = yahoo_financials.get_historical_price_data(start_date=start_date.strftime('%Y-%m-%d'), end_date=end_date.strftime('%Y-%m-%d'), time_interval='daily')
    return(data)

In [12]:
def getStock(npa):
    #position = npa[0]
    symbol = npa[0]
    start_=npa[1]
    end_=npa[2]
    #print(symbol)
    
    #tickerdata.history(period="2Y")
    #tickerdata = yf.Ticker(symbol)
    #data_ = tickerdata.history(start=start_,end=end_)
    data_ = yf.download(symbol, start=start_,end=end_)
    #data_['Sector'] = tickerdata.info['sector']
    #data_['Symbol'] = symbol
    #data_['Market Cap'] = data.get_quote_yahoo(names[n])['marketCap']
    #data_['Sector'] = tickerdata.info['sector']
    
    """
    tickerdata.info
    tickerdata.quarterly_balance_sheet
    tickerdata.quarterly_cashflow
    tickerdata.quarterly_dividends
    tickerdata.quarterly_earnings
    tickerdata.quarterly_financials
    tickerdata.actions
    tickerdata.mutualfund_holders
    tickerdata.options
    tickerdata.sustainability
    """
    
    return([symbol,data_])

In [13]:
def getTickerData(npa):
    symbol = npa[0]
    
    stock_data = yf.Ticker(symbol) 
    return([symbol,stock_data])

In [14]:
def getMarketData(npa):
    symbol = npa[0]

    #start_=npa[2]
    #end_=npa[3]
    #print(symbol)
    
    #tickerdata.history(period="2Y")
    #tickerdata = yf.Ticker(symbol)
    #data_ = tickerdata.history(start=start_,end=end_)
    #data_ = data.get_quote_yahoo(symbol)['marketCap']
    data_ = data.get_quote_yahoo(symbol)
    #data_['Sector'] = tickerdata.info['sector']
    
    """
    tickerdata.info
    tickerdata.quarterly_balance_sheet
    tickerdata.quarterly_cashflow
    tickerdata.quarterly_dividends
    tickerdata.quarterly_earnings
    tickerdata.quarterly_financials
    tickerdata.actions
    tickerdata.mutualfund_holders
    tickerdata.options
    tickerdata.sustainability
    """
    
    return([symbol,data_])

In [15]:
def getSector(npa):
    #position = npa[0]
    symbol = npa[0]
    #start_=npa[2]
    #end_=npa[3]
    
    tickerdata = yf.Ticker(symbol)
    return([symbol,tickerdata.info['sector']])

In [16]:
def derive_price_supply_trends(npa_):
    name = npa_[0]
    data = npa_[1]
    data_ = data
    data_['Symbol'] = name
    """
    if(np.sum(duplicates==names_prices[n])>0):
        #potential duplicate
        if((np.sum((np.unique(prices_df['Symbol']))==n))>0):
            pass
        else:
            prices_df = pd.concat([prices_df,data_],axis=0)            
    else:
    """
    
    data_['trailing_1yr_max'] = (pd.DataFrame(data_)['High']).rolling(252).max()
    data_['trailing_1yr_min'] = (pd.DataFrame(data_)['Low']).rolling(252).min()
    
    data_['supply_trend_1yr'] = pd.DataFrame(data_)['Adj Close']-(pd.DataFrame(data_)['High']).rolling(252).min()
    
    data_['supply_trend_180d'] = pd.DataFrame(data_)['Adj Close']-(pd.DataFrame(data_)['High']).rolling(126).min()
    
    data_['supply_trend_90d'] = pd.DataFrame(data_)['Adj Close']-(pd.DataFrame(data_)['High']).rolling(63).min()
    
    data_['supply_trend_60d'] = pd.DataFrame(data_)['Adj Close']-(pd.DataFrame(data_)['High']).rolling(42).min()
    
    data_['supply_trend_30d'] = pd.DataFrame(data_)['Adj Close']-(pd.DataFrame(data_)['High']).rolling(21).min()

    data_['trailing_60d_max'] = (pd.DataFrame(data_)['High']).rolling(42).max()
    data_['trailing_60d_min'] = (pd.DataFrame(data_)['Low']).rolling(42).min()
    
    data_['trailing_30d_max'] = (pd.DataFrame(data_)['High']).rolling(21).max()
    data_['trailing_30d_min'] = (pd.DataFrame(data_)['Low']).rolling(21).min()

    data_['trailing_risk_max'] = (pd.DataFrame(data_)['High']).rolling(40).max()
    data_['trailing_risk_min'] = (pd.DataFrame(data_)['Low']).rolling(40).min()
    
    data_['risk_per_share'] = data_['trailing_risk_max']-data_['trailing_risk_min']
    
    data_['30d_vol'] = (pd.DataFrame(data_)['Volume']).rolling(21).sum()
    
    data_['momentum_1yr'] = (data_['trailing_1yr_max']-data_['trailing_1yr_min']) + (data_['trailing_60d_max']-data_['trailing_60d_min']) + (data_['trailing_30d_max']-data_['trailing_30d_min']) + (data_['trailing_risk_max']-data_['trailing_risk_min'])
    
    data_['risk_trend_factor'] = data_['momentum_1yr']/data_['risk_per_share']
    
    return(data_)

In [17]:
def extractTickerValues(npa):
    n_ = npa[0]
    r_ = npa[1]
    #print(r_)
    
    subset = pd.DataFrame([r_.info])
    #print(subset)
    
    if(np.sum(np.where(subset.columns=='beta'))==0):
        b_ = 'error'
    else:
        b_ = subset['beta'].values[0]
        
    if(np.sum(np.where(subset.columns=='returnOnAssets'))==0):
        roa_ = 'error'
    else:
        roa_ = subset['returnOnAssets'].values[0]
                
    if(np.sum(np.where(subset.columns=='returnOnEquity'))==0):
        roe_ = 'error'
    else:
        roe_ = subset['returnOnEquity'].values[0]
        
    if(np.sum(np.where(subset.columns=='debtToEquity'))==0):
        debtToEquity = 'error'
    else:
        debtToEquity = subset['debtToEquity'].values[0]
        
    if(np.sum(np.where(subset.columns=='quickRatio'))==0):
        quickRatio = 'error'
    else:
        quickRatio = subset['quickRatio'].values[0]
        
    if(np.sum(np.where(subset.columns=='dividendRate'))==0):
        dividendRate = 'error'
    else:        
        dividendRate = subset['dividendRate'].values[0]    

    data_ = pd.DataFrame([[b_,roa_,roe_,quickRatio,debtToEquity,dividendRate]],columns=['beta','roa','roe','debtToEquity','quickRatio','dividendDate'],index=['n_'])
    
    return([n_,data_])

In [18]:
def calculateMetrics(data_):
    data_df = data_
    n_ = data_df['Symbol'].values[-1]
    
    latest_p = pd.DataFrame(data_df)['Close'][-1]
    oneYearDate = pd.DataFrame(data_df)['Close'][[-1]].index[0] - relativedelta(years=1)
    
    #separate subset for past year.
    past_year = data_df[data_df.index>=oneYearDate.strftime('%Y-%m-%d')]
    
    #np.max(npa[np.where(np.array(names_prices)=='GBX')[0][0]]['High'])
    max_2y = pd.DataFrame(data_df)['High'].rolling(504).max()[-1]
    min_2y = pd.DataFrame(data_df)['Low'].rolling(504).min()[-1]
    
    #quantiles_close_2y = data_df['Close'].rolling(504).quantile(q=[0, .02, .09, .25, .5, .75, .91, .98, 1])[-1]
    quantiles_close_2y = data_df['Close'].quantile(q=[0, .02, .09, .25, .5, .75, .91, .98, 1])
    
    std_2y = pd.DataFrame(data_df)['Close'].rolling(504).std()[-1]
    mean_2y = pd.DataFrame(data_df)['Close'].rolling(504).mean()[-1]
    
    max_1y = past_year['High'].rolling(252).max()[-1]
    min_1y = past_year['Low'].rolling(252).min()[-1]
    
    #quantiles_close_1y = data_df['Close'].rolling(252).quantile(q=[0, .02, .09, .25, .5, .75, .91, .98, 1])[-1]
    quantiles_close_1y = past_year['Close'].quantile(q=[0, .02, .09, .25, .5, .75, .91, .98, 1])
    
    std_1y = past_year['Close'].rolling(252).std()[-1]
    mean_1y = past_year['Close'].rolling(252).mean()[-1]
    
    risk_trend = data_df['risk_per_share'][-1]
    
    supply_trend_1yr = data_df['supply_trend_1yr'][-1]
    supply_trend_180d = data_df['supply_trend_180d'][-1]
    supply_trend_90d = data_df['supply_trend_90d'][-1]
    supply_trend_60d = data_df['supply_trend_60d'][-1]
    supply_trend_30d = data_df['supply_trend_30d'][-1]
    
    #vol_30d_2yr = data_df['30d_vol'].rolling(504).mean()[-1]
    vol_30d_2yr = np.mean(data_df['30d_vol'])
    
    pct_2yr = (latest_p - min_2y)/(max_2y-min_2y)
    pct_1yr = (latest_p - min_1y)/(max_1y-min_1y)
    
    return([n_,max_2y,min_2y,std_2y,mean_2y,max_1y,min_1y,std_1y,mean_1y,latest_p,*quantiles_close_2y,*quantiles_close_1y,risk_trend,supply_trend_1yr,supply_trend_180d,supply_trend_90d,supply_trend_60d,supply_trend_30d,vol_30d_2yr,pct_1yr,pct_2yr])

In [19]:
npa = []
for i in range(0,len(SP1500)):
    npa.append([SP1500[i],start.strftime('%Y-%m-%d'),end.strftime('%Y-%m-%d')])
    
# = getStock(npa[0])
    
client = Client('192.168.3.100:8786')
client.restart()
future = client.map(getStock, npa)
results_prices = []
#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        results_prices.append("error")
    else:
        results_prices.append(f.result()) 
client.close()


C:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\distributed\client.py:1265: VersionMismatchWarning: Mismatched versions found

+-------------+---------------+---------------+---------------+
| Package     | client        | scheduler     | workers       |
+-------------+---------------+---------------+---------------+
| dask        | 2022.01.1     | 2021.11.2     | 2021.11.2     |
| distributed | 2022.01.1     | 2021.11.2     | 2021.11.2     |
| lz4         | 3.1.3         | 3.1.10        | 3.1.10        |
| python      | 3.9.7.final.0 | 3.9.9.final.0 | 3.9.9.final.0 |
| toolz       | 0.11.1        | 0.11.2        | 0.11.2        |
+-------------+---------------+---------------+---------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [20]:
npa = []
for i in range(0,len(SP1500)):
    npa.append([SP1500[i]])

#getSector(npa[0])

client = Client('192.168.3.100:8786')
client.restart()

future = client.map(getSector, npa)
results_sectors = []
#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        results_sectors.append("error")
    else:
        results_sectors.append(f.result()) 
client.close()


In [21]:
len(np.unique([r[0] for r in results_sectors]))

1172

In [22]:
#Drop Down screen metrics
#by sector
#further subdivide by market
#https://stackoverflow.com/questions/64919511/dateformatter-is-bringing-1970-as-year-not-the-original-year-in-the-dataset


In [23]:
sectors_df = pd.DataFrame()

for r in range(0,len(results_sectors)):
    r_ = results_sectors[r]
    if(r_=='error'):
        pass#sectors_df = pd.concat([sectors_df,pd.DataFrame([np.nan],index=[i_],columns=['Sector'])],axis=0)
    else:
        s_ = np.array(r_)[1]
        i_ = np.array(r_)[0]
        sectors_df = pd.concat([sectors_df,pd.DataFrame([s_],index=[i_],columns=['Sector'])],axis=0)

missing_sectors = list(set(SP1500).difference([r[0] for r in results_sectors]))

for r in range(0,len(missing_sectors)):
    r_ = missing_sectors[r]
    #s_ = np.array(r_)[1]
    #i_ = np.array(r_)[0]
    sectors_df = pd.concat([sectors_df,pd.DataFrame(['error'],index=[r_],columns=['Sector'])],axis=0)

In [24]:
#sectors_df

In [25]:
#mktcap_df

In [26]:
#np.sum(sectors_df.index.duplicated())

In [27]:
#sectors_df.reindex(mktcap_df.index.duplicated())

In [28]:
npa = []
for i in range(0,len(SP1500)):
    npa.append([SP1500[i]])

#getMarketCap(npa[0])

client = Client('192.168.3.100:8786')
client.restart()

future = client.map(getMarketData, npa)
results_mkt_data = []
#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        results_mkt_data.append("error")
    else:
        results_mkt_data.append(f.result()) 
client.restart()
client.close()


In [29]:
#results_mkt_data = results_mktcp

In [30]:
results_mkt_data[0][1].columns

Index(['language', 'region', 'quoteType', 'triggerable', 'quoteSourceName',
       'currency', 'priceHint', 'postMarketChangePercent', 'postMarketTime',
       'postMarketPrice', 'postMarketChange', 'regularMarketChangePercent',
       'regularMarketDayRange', 'regularMarketPreviousClose', 'bid', 'ask',
       'bidSize', 'askSize', 'messageBoardId', 'fullExchangeName', 'longName',
       'financialCurrency', 'averageDailyVolume3Month',
       'averageDailyVolume10Day', 'fiftyTwoWeekLowChange',
       'fiftyTwoWeekLowChangePercent', 'fiftyTwoWeekRange',
       'fiftyTwoWeekHighChange', 'fiftyTwoWeekHighChangePercent',
       'fiftyTwoWeekLow', 'fiftyTwoWeekHigh', 'earningsTimestamp',
       'earningsTimestampStart', 'earningsTimestampEnd', 'trailingPE',
       'epsTrailingTwelveMonths', 'epsForward', 'epsCurrentYear',
       'priceEpsCurrentYear', 'sharesOutstanding', 'bookValue',
       'fiftyDayAverage', 'fiftyDayAverageChange',
       'fiftyDayAverageChangePercent', 'twoHundredDayAve

In [31]:
#results_mktcp[10][1].columns

In [32]:
#np.array([r[0] for r in results_mktcp])
#names_mktcap = [n[0] for n in results_mktcp]

mktcap_df = pd.DataFrame()

for r in range(0,len(results_mkt_data)):
    r_ = results_mkt_data[r]
    if(r_=='error'):
        pass#sectors_df = pd.concat([sectors_df,pd.DataFrame([np.nan],index=[i_],columns=['Sector'])],axis=0)
    else:
        if(np.sum(np.where(np.array(r_[1].columns)=='marketCap'))==0):
            pass
        else:
            m_ = r_[1]['marketCap'][0]
            #print(s_[0])
            i_ = np.array(r_)[0]
            mktcap_df = pd.concat([mktcap_df,pd.DataFrame([m_],index=[i_],columns=['Market Cap'])],axis=0)



C:\Users\User\AppData\Local\Temp/ipykernel_26608/1182025761.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  i_ = np.array(r_)[0]


In [33]:
missing_mktcaps = list(set(SP1500).difference([n for n in mktcap_df.index]))

for r in range(0,len(missing_mktcaps)):
    r_ = missing_mktcaps[r]
    #s_ = np.array(r_)[1]
    #i_ = np.array(r_)[0]
    mktcap_df = pd.concat([mktcap_df,pd.DataFrame([np.nan],index=[r_],columns=['Market Cap'])],axis=0)

In [34]:
mktcap_df

,Market Cap
PCRX,2.864657e+09
ARW,8.491281e+09
VTR,2.242171e+10
SAIC,5.016537e+09
SYNH,7.651433e+09
...,...
JW-A,NaN
ENOV,NaN
OMCL,NaN
ZIMV,NaN


In [35]:
pe_trailing_df = pd.DataFrame()
    
npa = []
for r in range(0,len(results_mkt_data)):
    r_ = results_mkt_data[r]
    if(r_=='error'):
        pass
    else:
        if(np.sum(np.where(r_[1].columns=='trailingPE'))==0):
            pass
        else:
            pe_trailing_df = pd.concat([pe_trailing_df,pd.DataFrame(r_[1]['trailingPE'])],axis=0)
            
missing_pe_trailing = list(set(SP1500).difference([n for n in pe_trailing_df.index]))

for r in range(0,len(missing_pe_trailing)):
    r_ = missing_pe_trailing[r]
    #s_ = np.array(r_)[1]
    #i_ = np.array(r_)[0]
    pe_trailing_df = pd.concat([pe_trailing_df,pd.DataFrame([np.nan],index=[r_],columns=['trailingPE'])],axis=0)
            


In [36]:
pe_trailing_df

,trailingPE
PCRX,47.391304
ARW,9.378605
VTR,106.181470
SAIC,17.421394
SYNH,30.991180
...,...
RES,NaN
DVAX,NaN
RMAX,NaN
CIR,NaN


In [37]:
pe_forward_df = pd.DataFrame()
    
npa = []
for r in range(0,len(results_mkt_data)):
    r_ = results_mkt_data[r]
    if(r_=='error'):
        pass
    else:
        if(np.sum(np.where(r_[1].columns=='forwardPE'))==0):
            pass
        else:
            pe_forward_df = pd.concat([pe_forward_df,pd.DataFrame(r_[1]['forwardPE'])],axis=0)
            
missing_pe_forward = list(set(SP1500).difference([n for n in pe_forward_df.index]))

for r in range(0,len(missing_pe_forward)):
    r_ = missing_pe_forward[r]
    #s_ = np.array(r_)[1]
    #i_ = np.array(r_)[0]
    pe_forward_df = pd.concat([pe_forward_df,pd.DataFrame([np.nan],index=[r_],columns=['forwardPE'])],axis=0)
            


In [38]:
pe_forward_df

,forwardPE
PCRX,16.117794
ARW,7.532427
VTR,2808.500000
SAIC,12.270195
SYNH,14.468628
...,...
RILY,NaN
OMCL,NaN
SENEA,NaN
ZIMV,NaN


In [39]:
npa = []
for i in range(0,len(SP1500)):
    npa.append([SP1500[i]])

#getMarketCap(npa[0])

client = Client('192.168.3.100:8786')
client.restart()

future = client.map(getTickerData, npa)

results_ticker_data = []

#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        results_ticker_data.append("error")
    else:
        results_ticker_data.append(f.result()) 
client.restart()
client.close()


C:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\distributed\client.py:1265: VersionMismatchWarning: Mismatched versions found

+-------------+---------------+---------------+---------------+
| Package     | client        | scheduler     | workers       |
+-------------+---------------+---------------+---------------+
| dask        | 2022.01.1     | 2021.11.2     | 2021.11.2     |
| distributed | 2022.01.1     | 2021.11.2     | 2021.11.2     |
| lz4         | 3.1.3         | 3.1.10        | 3.1.10        |
| python      | 3.9.7.final.0 | 3.9.9.final.0 | 3.9.9.final.0 |
| toolz       | 0.11.1        | 0.11.2        | 0.11.2        |
+-------------+---------------+---------------+---------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [40]:
print(results_ticker_data[0][1].info)
print(results_ticker_data[0][1].quarterly_balance_sheet)
print(results_ticker_data[0][1].quarterly_cashflow)
#print(results_ticker_data[0][1].quarterly_dividends)
print(results_ticker_data[0][1].quarterly_earnings)
print(results_ticker_data[0][1].quarterly_financials)


{'zip': '75201', 'sector': 'Real Estate', 'fullTimeEmployees': 105000, 'longBusinessSummary': 'CBRE Group, Inc. operates as a commercial real estate services and investment company worldwide. It operates through three segments: Advisory Services, Global Workplace Solutions, and Real Estate Investments segments. The Advisory Services segment provides strategic advice and execution to owners, investors, and occupiers of real estate in connection with leasing; property sales and mortgage services under the CBRE Capital Markets brand; property and project management services, including construction management, marketing, building engineering, accounting, and financial services for owners of and investors in office, industrial, and retail properties; and valuation services that include market value appraisals, litigation support, discounted cash flow analyses, and feasibility studies, as well as consulting services, such as property condition reports, hotel advisory, and environmental consu

In [41]:
results_ticker_data[0][1].info

{'zip': '75201',
 'sector': 'Real Estate',
 'fullTimeEmployees': 105000,
 'longBusinessSummary': 'CBRE Group, Inc. operates as a commercial real estate services and investment company worldwide. It operates through three segments: Advisory Services, Global Workplace Solutions, and Real Estate Investments segments. The Advisory Services segment provides strategic advice and execution to owners, investors, and occupiers of real estate in connection with leasing; property sales and mortgage services under the CBRE Capital Markets brand; property and project management services, including construction management, marketing, building engineering, accounting, and financial services for owners of and investors in office, industrial, and retail properties; and valuation services that include market value appraisals, litigation support, discounted cash flow analyses, and feasibility studies, as well as consulting services, such as property condition reports, hotel advisory, and environmental co

In [42]:
npa = []
results_ticker_names = [r[0] for r in results_ticker_data]
for n in range(0,len(results_ticker_names),1):
    npa.append([results_ticker_names[n],results_ticker_data[n][1]])

In [43]:

client = Client('192.168.3.100:8786')
client.restart()

future = client.map(extractTickerValues, npa)

results_ticker_values = []

#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        results_ticker_values.append("error")
    else:
        results_ticker_values.append(f.result()) 
client.restart()
client.close()



C:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\distributed\client.py:1265: VersionMismatchWarning: Mismatched versions found

+-------------+---------------+---------------+---------------+
| Package     | client        | scheduler     | workers       |
+-------------+---------------+---------------+---------------+
| dask        | 2022.01.1     | 2021.11.2     | 2021.11.2     |
| distributed | 2022.01.1     | 2021.11.2     | 2021.11.2     |
| lz4         | 3.1.3         | 3.1.10        | 3.1.10        |
| python      | 3.9.7.final.0 | 3.9.9.final.0 | 3.9.9.final.0 |
| toolz       | 0.11.1        | 0.11.2        | 0.11.2        |
+-------------+---------------+---------------+---------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [44]:
ticker_values = pd.DataFrame()

for r in results_ticker_values:
    if(r=='error'):
        pass
    else:                
        #print(r[1])
        subset = pd.DataFrame(r[1])
        subset.index = [r[0]]
        #print(subset)
        ticker_values = pd.concat([ticker_values,subset],axis=0)
        #ticker_values = pd.concat([ticker_values,pd.DataFrame(r[1],index=[r[0]])],axis=0)

In [45]:
market_data = pd.concat([mktcap_df,ticker_values,sectors_df.loc[~sectors_df.index.duplicated(), :].reindex(mktcap_df.loc[~mktcap_df.index.duplicated(), :].index)],axis=1)

In [46]:
market_data.sort_values(by='Market Cap',ascending=False)

,Market Cap,beta,roa,roe,debtToEquity,quickRatio,dividendDate,Sector
AAPL,2.385163e+12,1.194642,0.21695,1.49271,0.76,178.016,0.92,Technology
MSFT,2.027305e+12,0.93616,0.15674,0.48721,1.773,47.863,2.48,Technology
GOOG,1.520775e+12,1.132142,0.15049,0.308,2.738,11.26,None,Communication Services
GOOGL,1.520649e+12,1.132142,0.15049,0.308,2.738,11.26,None,Communication Services
AMZN,1.240991e+12,1.231696,0.03395,0.18046,0.707,109.9,None,Consumer Cyclical
...,...,...,...,...,...,...,...,...
JW-A,NaN,error,error,error,error,error,error,error
ENOV,NaN,error,error,error,error,error,error,error
OMCL,NaN,error,error,error,error,error,error,Healthcare
ZIMV,NaN,None,-0.0242,-0.06479,1.433,3.989,None,Healthcare


In [47]:
market_data

,Market Cap,beta,roa,roe,debtToEquity,quickRatio,dividendDate,Sector
PCRX,2.864657e+09,error,error,error,error,error,error,Healthcare
ARW,8.491281e+09,1.369642,0.06213,0.2385,1.018,57.253,None,Technology
VTR,2.242171e+10,1.180803,0.01615,0.01424,0.306,113.533,1.8,Real Estate
SAIC,5.016537e+09,error,error,error,error,error,error,Technology
SYNH,7.651433e+09,error,error,error,error,error,error,Healthcare
...,...,...,...,...,...,...,...,...
JW-A,NaN,error,error,error,error,error,error,error
ENOV,NaN,error,error,error,error,error,error,error
OMCL,NaN,error,error,error,error,error,error,Healthcare
ZIMV,NaN,None,-0.0242,-0.06479,1.433,3.989,None,Healthcare


In [48]:
#total_mkt_cap = mktcap_df.replace([np.nan], 0).sum()
#mktcap_df['mkt_cap_pct'] = mktcap_df.replace([np.nan], 0)/total_mkt_cap

In [49]:
#[rp[2] for rp in results_prices][0].info['sector']

In [50]:
"""
arr=np.array(names_prices)
pd.Series(arr).value_counts()
set_ = pd.Series(arr).value_counts()[pd.Series(arr).value_counts()>1]
if(len(set_)==0):
    duplicates = []
    pass
else:
    duplicates = set_.index[0]
    """

'\narr=np.array(names_prices)\npd.Series(arr).value_counts()\nset_ = pd.Series(arr).value_counts()[pd.Series(arr).value_counts()>1]\nif(len(set_)==0):\n    duplicates = []\n    pass\nelse:\n    duplicates = set_.index[0]\n    '

In [51]:
#len(results)

In [52]:
names_prices = [rp[0] for rp in results_prices]

#def process_df(data_):
npa = []    
for n in range(0,len(names_prices)):
    #print(names_prices[n])
    n_ = names_prices[n]
    npa.append([n_,results_prices[n][1]])
    
prices_df = pd.DataFrame()

client = Client('192.168.3.100:8786')
client.restart()

future = client.map(derive_price_supply_trends, npa)
#results_sectors = []
#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        #results_sectors.append("error")
        pass
    else:
        #results_sectors.append(f.result()) 
        prices_df = pd.concat([f.result(),prices_df],axis=0)
client.close()

C:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\distributed\client.py:1265: VersionMismatchWarning: Mismatched versions found

+-------------+---------------+---------------+---------------+
| Package     | client        | scheduler     | workers       |
+-------------+---------------+---------------+---------------+
| dask        | 2022.01.1     | 2021.11.2     | 2021.11.2     |
| distributed | 2022.01.1     | 2021.11.2     | 2021.11.2     |
| lz4         | 3.1.3         | 3.1.10        | 3.1.10        |
| python      | 3.9.7.final.0 | 3.9.9.final.0 | 3.9.9.final.0 |
| toolz       | 0.11.1        | 0.11.2        | 0.11.2        |
+-------------+---------------+---------------+---------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [53]:
#np.unique(np.array(prices_df['Symbol'].values))

In [54]:
#sectors_df.reindex(mktcap_df.index.duplicated())

In [55]:
prices_df.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Symbol',
       'trailing_1yr_max', 'trailing_1yr_min', 'supply_trend_1yr',
       'supply_trend_180d', 'supply_trend_90d', 'supply_trend_60d',
       'supply_trend_30d', 'trailing_60d_max', 'trailing_60d_min',
       'trailing_30d_max', 'trailing_30d_min', 'trailing_risk_max',
       'trailing_risk_min', 'risk_per_share', '30d_vol', 'momentum_1yr',
       'risk_trend_factor'],
      dtype='object')

In [56]:
#[rp[0] for rp in results_prices]

In [57]:
npa = []
for s in names_prices:
    #print(s)
    subset = prices_df.iloc[np.where(prices_df['Symbol']==s)]
    if(len(subset)==0):
        pass
    else:
        npa.append(subset)
    
screenerMetrics = []

client = Client('192.168.3.100:8786')
client.restart()

future = client.map(calculateMetrics, npa)

#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        screenerMetrics.append("error")
    else:
        screenerMetrics.append(f.result()) 
client.close()

missing_screener_metrics = list(set(SP1500).difference([n for n in pd.DataFrame(screenerMetrics).set_index(0).index]))
print(missing_screener_metrics)

for r in range(0,len(missing_screener_metrics)):
    r_ = missing_screener_metrics[r]
    test = np.array([r_,*np.repeat('missing',len(pd.DataFrame(screenerMetrics).set_index(0).iloc[[0]].columns))])
    screenerMetrics.append(test)

pd.DataFrame(screenerMetrics)


['JW-A']


,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,EXR,228.839996,88.879997,39.594781,153.268413,228.839996,154.309998,16.709053,189.302341,178.300003,...,226.729996,54.419998,20.949997,6.440002,6.440002,6.440002,6.440002,16682716.735537,0.321884,0.638897
1,AVNS,53.610001,24.84,6.204857,36.226746,40.720001,24.84,2.924113,32.730556,28.08,...,40.450001,9.224998,2.209999,2.209999,2.209999,2.209999,2.209999,6898945.041322,0.20403,0.112617
2,LVS,66.769997,28.879999,8.80263,47.370119,56.889999,28.879999,5.642989,40.844801,34.959999,...,56.759998,10.67,4.269999,4.269999,4.269999,4.269999,4.269999,152512494.421488,0.217065,0.160465
3,OGE,42.740002,28.25,3.033101,34.284544,42.740002,32.490002,2.566566,36.531746,41.0,...,42.330002,4.68,8.02,5.029999,2.189999,1.66,1.639999,27000471.07438,0.830244,0.879917
4,CBRE,111.0,41.029999,20.414821,77.980119,111.0,76.349998,8.112137,94.423254,79.900002,...,110.300003,13.410004,1.239998,1.239998,1.239998,1.239998,1.239998,34395376.652893,0.102453,0.555524
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1500,WRLD,265.75,60.950001,49.096964,151.169187,265.75,123.379997,26.552616,192.520199,139.300003,...,259.0,86.500008,7.0,7.0,7.0,7.0,7.0,932993.38843,0.111821,0.382568
1501,STE,255.929993,146.119995,26.86395,202.11256,255.929993,190.860001,12.586968,224.321945,220.470001,...,252.919998,46.869995,25.110001,3.860001,3.860001,3.860001,3.860001,10713165.289256,0.455048,0.677079
1502,NSC,299.200012,166.0,30.955846,249.837262,299.200012,222.539993,15.016813,267.017698,244.660004,...,297.709991,54.110001,15.089996,15.089996,15.089996,15.089996,15.089996,26658582.85124,0.288547,0.590541
1503,INTC,68.489998,40.310001,5.571991,52.921448,58.419998,40.310001,4.002611,50.875873,43.389999,...,58.189999,8.59,1.5,1.5,1.5,1.5,1.5,677184002.68595,0.170072,0.109297


In [58]:
#list(set(SP1500).difference([n for n in pd.DataFrame(screenerMetrics).set_index(0).index]))

In [59]:
index_names = ['SP1500', 'SP500', 'SP600', 'SP400']
indexes = ['SPTM','SPY','SLY','MDYG']
sector_names = ['Communication Services ETF', 'Consumer Discretionary ETF', 'Consumer Staples ETF', 'Energy ETF', 'Financials ETF', 'Health Care ETF', 'Industrials ETF', 'Information Technology ETF', 'Materials ETF', 'Real Estate ETF', 'Utilities ETF']
sectors = ['VOX','VCR','VDC','VDE','VFH','VHT','VIS','VGT','VAW','VNQ','VPU']

In [60]:
npa = []
for i in range(0,len(indexes)):
    npa.append([indexes[i],start.strftime('%Y-%m-%d'),end.strftime('%Y-%m-%d')])
    
# = getStock(npa[0])
    
client = Client('192.168.3.100:8786')
client.restart()
future = client.map(getStock, npa)
results_indexes = []
#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        results_indexes.append("error")
    else:
        results_indexes.append(f.result()) 
client.close()


C:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\distributed\client.py:1265: VersionMismatchWarning: Mismatched versions found

+-------------+---------------+---------------+---------------+
| Package     | client        | scheduler     | workers       |
+-------------+---------------+---------------+---------------+
| dask        | 2022.01.1     | 2021.11.2     | 2021.11.2     |
| distributed | 2022.01.1     | 2021.11.2     | 2021.11.2     |
| lz4         | 3.1.3         | 3.1.10        | 3.1.10        |
| python      | 3.9.7.final.0 | 3.9.9.final.0 | 3.9.9.final.0 |
| toolz       | 0.11.1        | 0.11.2        | 0.11.2        |
+-------------+---------------+---------------+---------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [61]:
npa = []
for i in range(0,len(sectors)):
    npa.append([sectors[i],start.strftime('%Y-%m-%d'),end.strftime('%Y-%m-%d')])
    
# = getStock(npa[0])
    
client = Client('192.168.3.100:8786')
client.restart()
future = client.map(getStock, npa)
results_sectors = []
#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        results_sectors.append("error")
    else:
        results_sectors.append(f.result()) 
client.close()


In [62]:
#def process_df(data_):
npa = []    
for n in range(0,len(indexes)):
    #print()
    data=results_indexes[n][1]
    data['Symbol'] = indexes[n]
    npa.append([indexes[n],data])
    
prices_indexes = pd.DataFrame()

client = Client('192.168.3.100:8786')
client.restart()

future = client.map(derive_price_supply_trends, npa)
#results_sectors = []
#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        #results_sectors.append("error")
        pass
    else:
        #results_sectors.append(f.result()) 
        prices_indexes = pd.concat([f.result(),prices_indexes],axis=0)
client.close()



In [63]:

#def process_df(data_):
npa = []    
for n in range(0,len(sectors)):
    #print()
    data=results_sectors[n][1]
    data['Symbol'] = sectors[n]
    npa.append([sectors[n],data])
    
prices_sectors = pd.DataFrame()

client = Client('192.168.3.100:8786')
client.restart()

future = client.map(derive_price_supply_trends, npa)
#results_sectors = []
#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        #results_sectors.append("error")
        pass
    else:
        #results_sectors.append(f.result()) 
        prices_sectors = pd.concat([f.result(),prices_sectors],axis=0)
client.close()


In [64]:
npa = []
for s in indexes:
    subset = prices_indexes.iloc[np.where(prices_indexes['Symbol']==s)]
    if(len(subset)==0):
        pass
    else:
        npa.append(subset)
    
indexMetrics = []

client = Client('192.168.3.100:8786')
client.restart()

future = client.map(calculateMetrics, npa)

#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        indexMetrics.append("error")
    else:
        indexMetrics.append(f.result()) 
client.close()


In [65]:
npa = []
for s in sectors:
    subset = prices_sectors.iloc[np.where(prices_sectors['Symbol']==s)]
    if(len(subset)==0):
        pass
    else:
        npa.append(subset)
    
sectorMetrics = []

client = Client('192.168.3.100:8786')
client.restart()

future = client.map(calculateMetrics, npa)

#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        sectorMetrics.append("error")
    else:
        sectorMetrics.append(f.result()) 
client.close()


In [66]:
#market_data.index = [sub.replace('.', '-') for sub in market_data.index]


In [67]:
set(np.array(market_data.index)).difference(pd.DataFrame(screenerMetrics).set_index(0).index)

set()

In [68]:
metrics = pd.concat([pd.DataFrame(screenerMetrics),pd.DataFrame(indexMetrics),pd.DataFrame(sectorMetrics)],axis=0)

In [69]:
#np.min(npa[np.where(np.array(names_prices)=='GBX')[0][0]]['Open'])

In [70]:
#calculateMinMax(npa[np.where(np.array(names_prices)=='GBX')[0][0]])

In [71]:
#pd.DataFrame(screener['risk_trend']).sort_values(by='risk_trend',ascending=False)

In [72]:
prices_df = pd.concat([prices_df,prices_indexes,prices_sectors],axis=0)

In [73]:
npa = []
for f in FRED_Indicators:
    npa.append([f,start.strftime('%Y-%m-%d'),end.strftime('%Y-%m-%d')])

In [74]:
   
completed_fred = pd.DataFrame()

client = Client('192.168.3.100:8786')
client.restart()

future = client.map(Fred_Data, npa)

#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        pass
    else:
        completed_fred = pd.concat([completed_fred,f.result()],axis=0)
client.close()

completed_fred_pvt = pd.pivot_table(completed_fred, values=['Value'], index=['DATE'],columns=['Symbol'])
completed_fred_pvt

Value                                                   \
Symbol     ASPUS AWHAETP B432RG3A086NBEA BAA10Y BACDINA066MNFRBNY   
DATE                                                                
2020-06-05   NaN     NaN             NaN   2.86               NaN   
2020-06-06   NaN     NaN             NaN    NaN               NaN   
2020-06-07   NaN     NaN             NaN    NaN               NaN   
2020-06-08   NaN     NaN             NaN   2.81               NaN   
2020-06-09   NaN     NaN             NaN   2.80               NaN   
...          ...     ...             ...    ...               ...   
2022-05-30   NaN     NaN             NaN    NaN               NaN   
2022-05-31   NaN     NaN             NaN   2.18               NaN   
2022-06-01   NaN     NaN             NaN   2.08               NaN   
2022-06-02   NaN     NaN             NaN   2.10               NaN   
2022-06-03   NaN     NaN             NaN    NaN               NaN   

                                                                       \
Symbol     BACTSAMFRBDAL BAMLC0A0CM BAMLCC0A1AAATRIV BAMLCC0A4BBBTRIV   
DATE                                                                    
2020-06-05           NaN       1.60           766.77           938.56   
2020-06-06           NaN        NaN              NaN              NaN   
2020-06-07           NaN        NaN              NaN              NaN   
2020-06-08           NaN       1.56           769.10           943.74   
2020-06-09           NaN       1.59           771.40           945.70   
...                  ...        ...              ...              ...   
2022-05-30           NaN       1.41           680.42           886.65   
2022-05-31           NaN       1.40           676.42           883.75   
2022-06-01           NaN       1.37           674.43           881.33   
2022-06-02           NaN       1.38           673.89           881.84   
2022-06-03           NaN        NaN              NaN              NaN   

                        ...                                              \
Symbol     BAMLH0A3HYC  ... USROE USSTHPI VIXCLS VXVCLS WDFUELLA WGS3MO   
DATE                    ...                                               
2020-06-05       13.21  ...   NaN     NaN  24.52  27.42    1.124   0.15   
2020-06-06         NaN  ...   NaN     NaN    NaN    NaN      NaN    NaN   
2020-06-07         NaN  ...   NaN     NaN    NaN    NaN      NaN    NaN   
2020-06-08       13.16  ...   NaN     NaN  25.81  28.08      NaN    NaN   
2020-06-09       13.29  ...   NaN     NaN  27.57  29.67      NaN    NaN   
...                ...  ...   ...     ...    ...    ...      ...    ...   
2022-05-30        9.79  ...   NaN     NaN  26.54    NaN      NaN    NaN   
2022-05-31        9.83  ...   NaN     NaN  26.19  27.99      NaN    NaN   
2022-06-01        9.70  ...   NaN     NaN  25.69  27.72      NaN    NaN   
2022-06-02        9.70  ...   NaN     NaN  24.72  27.06      NaN    NaN   
2022-06-03         NaN  ...   NaN     NaN    NaN    NaN      NaN    NaN   

                                                        
Symbol     WILLLRGCAPVAL WILLMICROCAPPR WPU0911 WTB3MS  
DATE                                                    
2020-06-05      90778.48       11717.89     NaN   0.15  
2020-06-06           NaN            NaN     NaN    NaN  
2020-06-07           NaN            NaN     NaN    NaN  
2020-06-08      92383.94       12070.97     NaN    NaN  
2020-06-09      90990.35       11984.24     NaN    NaN  
...                  ...            ...     ...    ...  
2022-05-30           NaN            NaN     NaN    NaN  
2022-05-31     123329.51       13744.24     NaN    NaN  
2022-06-01     122355.57       13646.47     NaN    NaN  
2022-06-02     123713.56       13914.28     NaN    NaN  
2022-06-03           NaN            NaN     NaN    NaN  

[729 rows x 130 columns]

In [75]:
npa = []
for e in etf_bonds:
    npa.append([e,start.strftime('%Y-%m-%d'),end.strftime('%Y-%m-%d')])
    
completed_bonds = pd.DataFrame()

client = Client('192.168.3.100:8786')
client.restart()

future = client.map(dl, npa)

#my intent was to capture future objects vs results and this gave me results
for f in as_completed(future):
    if(f.status=="error"):
        pass
    else:
        completed_bonds = pd.concat([completed_bonds,f.result()],axis=0)
client.close()

completed_bonds_pvt = pd.pivot_table(completed_bonds, values='Close', index=['Date'],columns=['Symbol'])
completed_bonds_pvt

C:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\distributed\client.py:1265: VersionMismatchWarning: Mismatched versions found

+-------------+---------------+---------------+---------------+
| Package     | client        | scheduler     | workers       |
+-------------+---------------+---------------+---------------+
| dask        | 2022.01.1     | 2021.11.2     | 2021.11.2     |
| distributed | 2022.01.1     | 2021.11.2     | 2021.11.2     |
| lz4         | 3.1.3         | 3.1.10        | 3.1.10        |
| python      | 3.9.7.final.0 | 3.9.9.final.0 | 3.9.9.final.0 |
| toolz       | 0.11.1        | 0.11.2        | 0.11.2        |
+-------------+---------------+---------------+---------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


Symbol,AGG,AGZ,BIL,GBF,GOVT,IEF,IEI,ILTB,IMTB,ISTB,IUSB,LQD,MEAR,MUB,NEAR,SHV,SHY,SUB,TLT
Date,,,,,,,,,,,,,,,,,,,
2020-06-05,114.522629,118.028030,91.516998,121.541183,27.040316,118.710236,131.382507,70.084915,50.745583,50.346340,52.538898,128.774750,49.764549,112.023109,49.205063,110.498024,86.088745,106.506859,153.706177
2020-06-08,114.649971,117.861198,91.527000,121.955223,27.059935,118.839096,131.451889,70.499634,50.892899,50.316944,52.607212,129.203125,49.774494,112.101418,49.214954,110.498024,86.068848,106.457375,154.296936
2020-06-09,114.796921,118.086929,91.527000,122.024231,27.167822,119.265274,131.719513,70.750366,50.829491,50.326748,52.626720,128.930511,49.784447,112.444061,49.234745,110.498024,86.098709,106.734375,156.049530
2020-06-10,115.267159,118.548164,91.537003,122.605865,27.285515,120.157333,132.205215,71.512268,50.907532,50.414936,52.870682,129.777573,49.804348,112.493004,49.294090,110.498024,86.148468,106.813522,158.363342
2020-06-11,115.071236,118.450012,91.516998,122.576286,27.413017,120.583504,132.244858,71.194000,50.985577,50.297348,52.685276,128.229462,49.704857,112.659431,49.224846,110.517990,86.108650,106.714600,161.356522
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-27,104.129997,111.230003,91.459999,110.150002,24.219999,104.410004,120.919998,58.549999,46.099998,48.150002,48.180000,115.290001,49.750000,108.279999,49.459999,110.269997,83.480003,104.779999,119.080002
2022-05-31,103.470001,110.940002,91.449997,109.370003,24.129999,103.320000,120.320000,57.930000,45.790001,48.029999,47.820000,114.440002,49.759998,108.099998,49.450001,110.239998,83.349998,104.669998,116.559998
2022-06-01,102.860001,110.309998,91.419998,108.690002,24.040001,102.610001,119.680000,57.439999,45.480000,47.820000,47.599998,113.500000,49.740002,107.940002,49.389999,110.169998,83.129997,104.660004,116.220001


In [76]:
#return([n_,max_2y,min_2y,std_2y,mean_2y,max_1y,min_1y,std_1y,mean_1y,latest_p,*quantiles_close_2y,*quantiles_close_1y,risk_trend,supply_trend_1yr,supply_trend_180d,supply_trend_90d,supply_trend_60d,supply_trend_30d,vol_30d_2yr,pct_1yr,,pct_2yr])
metrics.columns = ['Symbol','max_2y','min_2y','std_2y','mean_2y','max_1y','min_1y','std_1y','mean_1y','latest_p','2yr_close_0pct','2yr_close_2pct','2yr_close_9pct','2yr_close_25pct','2yr_close_50pct','2yr_close_75pct','2yr_close_91pct','2yr_close_98pct','2yr_close_100pct','1yr_close_0pct','1yr_close_2pct','1yr_close_9pct','1yr_close_25pct','1yr_close_50pct','2yr_close_75pct','1yr_close_91pct','1yr_close_98pct','1yr_close_100pct','risk_trend','supply_trend_1yr','supply_trend_180d','supply_trend_90d','supply_trend_60d','supply_trend_30d','vol_30d_2yr','pct_1yr','pct_2yr']

In [77]:
completed_bonds

,Open,High,Low,Close,Volume,Symbol
Date,,,,,,
2020-06-05,112.160164,112.160164,111.680473,112.023109,1916000,MUB
2020-06-08,112.111202,112.218889,112.042674,112.101418,1362900,MUB
2020-06-09,112.267847,112.561540,112.189528,112.444061,3125800,MUB
2020-06-10,112.512580,112.571315,112.355942,112.493004,1458600,MUB
2020-06-11,112.571321,112.825854,112.551745,112.659431,951100,MUB
...,...,...,...,...,...,...
2022-05-27,104.449997,104.589996,104.220001,104.410004,2508200,IEF
2022-05-31,103.620003,103.629997,103.150002,103.320000,6773200,IEF
2022-06-01,103.269997,103.370003,102.419998,102.610001,5555900,IEF


In [78]:
screener = metrics
a = screener.set_index('Symbol')[screener.set_index('Symbol')['risk_trend'].apply(type) != np.str_].sort_values(by='risk_trend',ascending=False)
b = screener.set_index('Symbol')[screener.set_index('Symbol')['risk_trend'].apply(type) == np.str_]
screener_sorted = pd.concat([a,b],axis=0)


In [79]:
screener_sorted

,max_2y,min_2y,std_2y,mean_2y,max_1y,min_1y,std_1y,mean_1y,latest_p,2yr_close_0pct,...,1yr_close_100pct,risk_trend,supply_trend_1yr,supply_trend_180d,supply_trend_90d,supply_trend_60d,supply_trend_30d,vol_30d_2yr,pct_1yr,pct_2yr
Symbol,,,,,,,,,,,,,,,,,,,,,
AMZN,3773.080078,2025.199951,296.531055,3180.708332,3773.080078,2025.199951,375.036636,3205.775125,2447.0,2082.0,...,3731.409912,1176.380127,339.0,339.0,339.0,339.0,339.0,82062094.834711,0.241321,0.241321
GOOG,3042.0,1347.01001,517.905498,2259.353414,3042.0,2044.160034,201.916485,2704.086196,2291.280029,1359.900024,...,3014.179932,709.869995,163.380127,163.380127,163.380127,163.380127,163.380127,30572394.834711,0.247655,0.557095
GOOGL,3030.929932,1351.650024,511.86544,2244.401052,3030.929932,2037.689941,208.932014,2684.163652,2290.820068,1362.540039,...,2996.77002,705.600098,161.650146,161.650146,161.650146,161.650146,161.650146,34995544.628099,0.254853,0.55927
NVR,5982.450195,3050.0,592.039014,4608.963436,5982.450195,3998.0,416.209297,4986.021081,4428.0,3098.629883,...,5959.330078,694.600098,250.080078,250.080078,250.080078,250.080078,250.080078,462794.421488,0.216685,0.469914
AZO,2267.399902,1074.449951,339.630637,1527.495833,2267.399902,1371.140015,212.855434,1819.807737,2058.040039,1095.699951,...,2254.030029,564.079956,665.5,258.310059,236.910034,236.910034,236.910034,4200658.264463,0.766407,0.824502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NYMT,4.93,2.26,0.70006,3.699226,4.93,2.84,0.478171,3.934762,3.11,2.34,...,4.82,0.71,0.17,0.17,0.17,0.17,0.17,70848274.793388,0.129187,0.318352
TWO,8.15,4.62,0.840578,6.114583,8.15,4.71,0.765604,6.030079,5.33,4.7,...,8.09,0.69,0.52,0.52,0.52,0.52,0.27,99173650.206612,0.180233,0.201133
GNW,4.76,1.87,0.596799,3.673155,4.61,3.23,0.290099,3.916548,3.99,2.01,...,4.54,0.63,0.67,0.38,0.38,0.38,0.14,106843557.231405,0.550725,0.733564


In [80]:
writer = pd.ExcelWriter('SP1500_screener_multiple.xlsx', engine='xlsxwriter')

prices_df.to_excel(writer, sheet_name='SP1500_Sectors_Indexes_Prices')
#prices_df.drop(['Sector', 'Market Cap'], axis=1).to_excel(writer, sheet_name='SP1500Prices')

sectors_df.to_excel(writer, sheet_name='stock_sectors')

mktcap_df.to_excel(writer, sheet_name='market_caps')

sp1500_index_df.to_excel(writer, sheet_name='SP1500')

pd.DataFrame(sp500).to_excel(writer, sheet_name='SP500')
pd.DataFrame(sp600).to_excel(writer, sheet_name='SP600')
pd.DataFrame(sp400).to_excel(writer, sheet_name='SP400')
pd.DataFrame(market_data).to_excel(writer, sheet_name='Stock_Fundamentals')
completed_fred_pvt.to_excel(writer, sheet_name='completed_fred_pvt')
completed_bonds.to_excel(writer, sheet_name='completed_bonds')
completed_bonds_pvt.to_excel(writer, sheet_name='completed_bonds_pvt')
pd.DataFrame(pd.DataFrame(sectors,sector_names)).to_excel(writer, sheet_name='vanguard_sectors')
pd.DataFrame(pd.DataFrame(indexes,index_names)).to_excel(writer, sheet_name='spdr_indexes')
screener_sorted.to_excel(writer, sheet_name='Screener')

writer.save()

wb = openpyxl.load_workbook('SP1500_screener_multiple.xlsx')
#openpyxl.Workbook()
new_range = openpyxl.workbook.defined_name.DefinedName('Prices', attr_text='SP1500_Sectors_Indexes_Prices!$A$1:$H$'+str(len(prices_df)+1))
wb.defined_names.append(new_range)
new_range = openpyxl.workbook.defined_name.DefinedName('Sectors', attr_text='stock_sectors!$A$1:$B$'+str(len(sectors_df)+1))
wb.defined_names.append(new_range)
new_range = openpyxl.workbook.defined_name.DefinedName('market_caps', attr_text='market_caps!$A$1:$B$'+str(len(mktcap_df)+1))
wb.defined_names.append(new_range)
new_range = openpyxl.workbook.defined_name.DefinedName('SP1500', attr_text='SP1500!$B$1:$C$'+str(len(sp1500_index_df)+1))
wb.defined_names.append(new_range)
new_range = openpyxl.workbook.defined_name.DefinedName('Vanguard_Sectors', attr_text='vanguard_sectors!$B$1:$C$'+str(len(sectors)+1))
wb.defined_names.append(new_range)
new_range = openpyxl.workbook.defined_name.DefinedName('SPDR_Indexes', attr_text='spdr_indexes!$B$1:$C$'+str(len(indexes)+1))
wb.defined_names.append(new_range)


#wb.defined_names.append(new_range)
wb.save('SP1500_screener_multiple.xlsx')
wb.close()
#named_range = NamedRange("test_nr", [(new_sheet, "A1")])
#wb.add_named_range(new_range)
"""
workbook = xlsxwriter.Workbook('SP1500_screener_multiple.xlsx')
workbook.define_name('Prices', '=SP1500Prices!$A$1:$J$'+str(len(prices_df)+1))
workbook.define_name('Sectors', '=stock_sectors!$A$1:$B$'+str(len(sectors_df)+1))
workbook.define_name('market_caps', '=market_caps!$A$1:$B$'+str(len(mktcap_df)+1))
workbook.define_name('SP1500', '=SP1500!$B$1:$C$'+str(len(sp1500_index_df)+1))

workbook.close()
"""

"\nworkbook = xlsxwriter.Workbook('SP1500_screener_multiple.xlsx')\nworkbook.define_name('Prices', '=SP1500Prices!$A$1:$J$'+str(len(prices_df)+1))\nworkbook.define_name('Sectors', '=stock_sectors!$A$1:$B$'+str(len(sectors_df)+1))\nworkbook.define_name('market_caps', '=market_caps!$A$1:$B$'+str(len(mktcap_df)+1))\nworkbook.define_name('SP1500', '=SP1500!$B$1:$C$'+str(len(sp1500_index_df)+1))\n\nworkbook.close()\n"